In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import category_encoders as ce
import calendar
import pickle

from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./data/ship_clean.csv')

In [3]:
# set y-

y = df['ontime']

In [ ]:
# get a baseline (57%)-

y.value_counts(normalize=True)

## Create a new, stripped-down model to be pickled and use with Flask web application:

In [60]:
# new X, based on top 15 most important features from the best-performing ExtraTrees model-

| Feature |Importance | 
|------------|:--------:|
| Standard Class shipping mode | 0.082940 | 
| Estimated Ship Days  | 0.073920  | 
| First Class shipping mode | 0.055566  | 
| Latitude | 0.039178  | 
| Customer ID | 0.039177  | 
| Longitude | 0.038545  | 
| Order ID | 0.036335  | 
| Second Class shipping mode | 0.023595  | 
| Order Hour of Day | 0.021846  | 
| Order Profit Ratio | 0.021365  | 
| Order Discount Rate | 0.021216  | 
| Same Day shipping mode | 0.020075  | 
| Order before-noon | 0.019108  | 
| Order Day of Week | 0.017546  |
| Order Month | 0.017070  | 

In [8]:
X = df[['estimated_ship_days', 'latitude', 'longitude', 'customer_id', 'order_id', 'order_hour', 'order_item_profit_ratio', 'order_item_discount_rate', 'order_before_noon', 'order_weekday', 'order_month']] 

In [9]:
X.shape

(180519, 11)

In [10]:
y.shape

(180519,)

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   estimated_ship_days       180519 non-null  int64  
 1   latitude                  180519 non-null  float64
 2   longitude                 180519 non-null  float64
 3   customer_id               180519 non-null  int64  
 4   order_id                  180519 non-null  int64  
 5   order_hour                180519 non-null  int64  
 6   order_item_profit_ratio   180519 non-null  float64
 7   order_item_discount_rate  180519 non-null  float64
 8   order_before_noon         180519 non-null  int64  
 9   order_weekday             180519 non-null  int64  
 10  order_month               180519 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 15.1 MB


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [13]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [14]:
etc3 = ExtraTreesClassifier()

In [15]:
etc3.fit(X_train_sc, y_train)

ExtraTreesClassifier()

In [16]:
etc3.score(X_test_sc, y_test)

0.916175493020164

In [ ]:
# so even with these stripped-down features (and missing the shipping modes, 
# which were throwing an error in the Flask application as the only categorical
# feature) the Extra Trees model is still scoring in the low 90s% on test

In [17]:
etc3 = ExtraTreesClassifier()
etc3.fit(X_train_sc, y_train)

print(f'This model has an accuracy score of {100 * etc3.score(X_test_sc, y_test):.2f}% on unseen data.')

In [ ]:
with open('./models/demo2.pkl', 'wb') as pickle_out:
    pickle_out = pickle.dump(etc3, pickle_out)